In [ ]:
import requests
import pandas as pd
import numpy as np
import warnings
from google.colab import files
from IPython.display import clear_output
warnings.filterwarnings('ignore', message='Unverified HTTPS request')
pd.options.display.float_format = '{:,.4f}'.format

# Downloading Mercado Libre data to Excel files
---

In this notebook, the data will be extracted from the MercadoLibre API of the Cellular and Smartphones category. (https://developers.mercadolibre.com.co/es_ar/api-docs-es)

1. The data of the best-selling products of the category according to MercadoLibre will be extracted.
2. 1000 data will be extracted from the mentioned category, which is the query limit with the offset parameter (pager).
3. All possible data will be extracted for each phrase from the trends tool provided by MercadoLibre.
4. Extracted bases in 2 and 3 will be joined by removing duplicates.

> This database will be conformed of:
*  Product ID
*  Product name
*  Price
*  Sold quantity
*  Product condition
*  Free shipping (yes o no)
*  Brand
*  Rating average
*  Model

5.  Product reviews data will be extracted.

> This database will be conformed of:
*  Product ID
*  Review title
*  Content
*  Rating
*  Likes
*  Dislikes
*  Valorization



### 1. Best-seller extraction MercadoLibre

For this extraction we will use a token that is obtained by creating an application in a MercadoLibre account, with the application id, the client id and the url of the page chosen to create the application.

It should be noted that this token has an operating time limit, so a new token must be obtained again in the next call of the code.

In [ ]:
"""app_id='50043454303347' --> ID de la aplicación creada
client_secret='p9Jbx92slor9ZYGza46iU6Lr0Rov4BEn' --> ID del cliente
code='TG-630b885a27200300012b5912-1186469550' --> código obtenido al dar autorización al cliente de usar la cuenta de MercadoLibre
redirect_uri='https://github.com/GabiMath' --> Url usada para crear la aplicación """

access_token='APP_USR-50043454303347-082901-a501ae8d0eeaee1c56f95ce1c319dff6-1186469550' # Obtained using the above parameters in postman

# Create a function to get the data for each country
def best_seller_func(site_id, category_id):
    url = "https://api.mercadolibre.com/highlights/"+site_id+"/category/"+category_id # Url with country and category code 
    payload={}
    headers = {
        'Authorization': 'Bearer '+access_token
    }
    response = requests.request("GET", url, headers=headers, data=payload, verify=False)
    best_seller = pd.DataFrame(response.json()['content'])
    return best_seller

def best_seller_names(sell_array):
    names = []
    for i in range(len(sell_array)):
        url_item = "https://api.mercadolibre.com/"+array_venta['type'][i].lower()+"s/"+array_venta['id'][i]
        payload={}
        headers = {} # Empty because it doesn't need authorization
        response = requests.request("GET", url_item, headers=headers, data=payload, verify=False).json()
        price = response["buy_box_winner"]["price"] if array_venta['type'][i]=="PRODUCT" else response["price"]
        name = response["name"] if array_venta['type'][i]=="PRODUCT" else response["title"]
        names.append({"id": response['id'], "name": name, "sold_quantity": response["sold_quantity"], 
                  "price": price})
        return sell_array.merge(pd.DataFrame(names), on="id", how="left")

best_seller_co = best_seller_func('MCO', 'MCO1055')
best_seller_co = best_seller_names(best_seller_co)
best_seller_mx = best_seller_func('MLM', 'MLM1055')
best_seller_mx = best_seller_names(best_seller_mx)

## Best-sellers Colombia:

In [ ]:
best_seller_co

,id,position,type,name,sold_quantity,price
0,MCO18969685,1,PRODUCT,Xiaomi Pocophone Poco M4 Pro Dual SIM 128 GB p...,967,959900
1,MCO15927746,2,PRODUCT,Xiaomi Redmi 9A Dual SIM 32 GB gris granito 2 ...,5337,399900
2,MCO18968544,3,PRODUCT,Xiaomi Pocophone Poco X4 Pro 5G Dual SIM 256 G...,1280,1442000
3,MCO18924350,4,PRODUCT,Xiaomi Redmi Note 11 (Snapdragon) Dual SIM 128...,821,792064
4,MCO18706709,5,PRODUCT,Xiaomi Pocophone M4 Pro 5G Dual SIM 128 GB coo...,378,835900
5,MCO18983369,6,PRODUCT,Xiaomi Redmi Note 11S Dual SIM 128 GB azul oca...,334,989900
6,MCO18924349,7,PRODUCT,Xiaomi Redmi Note 11 (Snapdragon) Dual SIM 128...,934,809900
7,MCO18706705,8,PRODUCT,Xiaomi Pocophone M4 Pro 5G Dual SIM 128 GB pow...,412,939900
8,MCO18924351,9,PRODUCT,Xiaomi Redmi Note 11 (Snapdragon) Dual SIM 128...,313,789900
9,MCO18954201,10,PRODUCT,Realme 9i Dual SIM 128 GB prism black 6 GB RAM,590,799900


## Best-sellers México

In [ ]:
best_seller_mx

,id,position,type,name,sold_quantity,price
0,MLM16092737,1,PRODUCT,Xiaomi Redmi 9A Dual SIM 64 GB gris granito 4 ...,33453,"2,324.0700"
1,MLM16092736,2,PRODUCT,Xiaomi Redmi 9A Dual SIM 64 GB azul celeste 4 ...,32971,"2,305.0000"
2,MLM17493810,3,PRODUCT,Samsung Galaxy A52 128 GB negro sorprendente 6...,16345,"5,799.0000"
3,MLM19069554,4,PRODUCT,Xiaomi Redmi 10C Dual SIM 128 GB azul océano 4...,3779,"3,149.0000"
4,MLM18924352,5,PRODUCT,Xiaomi Redmi Note 11 (Snapdragon) Dual SIM 128...,4644,"4,007.0000"
5,MLM18924351,6,PRODUCT,Xiaomi Redmi Note 11 (Snapdragon) Dual SIM 128...,7258,"3,799.0000"
6,MLM16092738,7,PRODUCT,Xiaomi Redmi 9A Dual SIM 64 GB verde majestuos...,14174,"2,335.0000"
7,MLM19069553,8,PRODUCT,Xiaomi Redmi 10C Dual SIM 128 GB gris grafito ...,4935,"3,099.0000"
8,MLM18597564,9,PRODUCT,Moto E40 Dual SIM 64 GB gris acero 4 GB RAM,9066,"2,870.0000"
9,MLM1465066893,10,ITEM,Celular Poco C40 Power Black 4gb Ram 64gb Rom,500,"2,960.0000"


As we can see, we have the best-selling products according to MercadoLibre, however, it is not consistent with the quantity sold attribute, so we will use the full base to do this analysis as well.

# 2. Smartphones category data extraction

For this extraction, we take the data of the category items and separately obtain the rating (through the reviews link) and the attributes. This upload takes approximately 5 minutes.

In [ ]:
def get_rating(ids):
    # La marca está dentro de los atributos en el diccionario, la extraigo con un ciclo con su id
    url_base = "https://api.mercadolibre.com/reviews/item/"
    payload={}
    headers = {}
    df = pd.DataFrame()
    response_rating = []
    for i in range(0, len(ids)):
        url = url_base+ids[i]
        response = requests.request("GET", url, headers=headers, data=payload, verify=False).json()
        response_rating.append({"id":ids[i], 
                               "rating":response['rating_average']})
        clear_output(wait=True)
        print("Extracting rating: "+str((i/len(ids))*100)+"%")
    return pd.DataFrame(response_rating)

def get_atributes2(ids):
    url_base = "https://api.mercadolibre.com/items/"
    payload={}
    headers = {}
    df = pd.DataFrame()
    response_att = []
    for i in range(0, len(ids)):
        url = url_base+ids[i]
        response = requests.request("GET", url, headers=headers, data=payload, verify=False).json()
        response_attributes = {}
        response_attributes['id']=response['id']
        df_at = pd.DataFrame(response['attributes'])
        for j in range(len(response['attributes'])):
            response_attributes[df_at['name'][j]]=df_at['value_name'][j]
        shipping = 1 if response['shipping']['free_shipping'] else 0
        response_attributes['free_shipping']=shipping
        response_att.append(response_attributes)
    return pd.DataFrame(response_att)

def get_atributes(response):
    response_att = []
    for i in range(len(response['results'])):
        response_attributes = {}
        response_attributes['id']=response['results'][i]['id']
        df_at = pd.DataFrame(response['results'][i]['attributes'])
        for j in range(len(response['results'][i]['attributes'])):
            response_attributes[df_at['name'][j]]=df_at['value_name'][j]
        shipping = 1 if response['results'][i]['shipping']['free_shipping'] else 0
        response_attributes['free_shipping']=shipping
        response_att.append(response_attributes)
    return pd.DataFrame(response_att)

def get_df(site_id, category_id, tipo):
    # Get data with requests 
    url_base = "https://api.mercadolibre.com/sites/"+site_id+"/search?"+tipo+"="+category_id
    payload={}
    headers = {}
        
    # Loop to get 50 more data until the offset limit (1000)
    df = pd.DataFrame()
    for i in range(0, 20):
        url = url_base+"&offset="+str(i*50)
        response = requests.request("GET", url, headers=headers, data=payload, verify=False).json()
        response_df = pd.DataFrame(response['results'])
        if 'category_id' in response_df.columns:
            response_df = response_df[response_df['category_id']==site_id+'1055']
        else:
            continue
        response_df = response_df[['id','title', 'price', 'original_price', 'sold_quantity', 'attributes', 'shipping']]
        marca = get_atributes2(response_df['id'])
        rating = get_rating(response_df['id'])
        response_df = response_df.merge(marca, on="id", how="left")
        response_df = response_df.merge(rating, on="id", how="left")
        df1 = pd.DataFrame(response_df)[['id','title', 'price', 'original_price','sold_quantity', 'Marca', 
                                         'Condición del ítem', 'Modelo', 'free_shipping', 'rating']]
        df = pd.concat([df, df1])
        clear_output(wait=True)
        print("General load process: "+str((i/20)*100)+"%")
    df = df.drop_duplicates()
    return df

In [ ]:
items_co = get_df("MCO", "MCO1055", "category")
items_mx = get_df("MLM", "MLM1055", "category")

Proceso de carga general: 95.0%


In [ ]:
items_co

,id,title,price,original_price,sold_quantity,Marca,Condición del ítem,Modelo,free_shipping,rating
0,MCO903059404,Xiaomi Pocophone Poco X4 Pro 5g Dual Sim 256 G...,1419900,NaN,58,Xiaomi,Nuevo,Poco X4 Pro 5G,1,4.8000
1,MCO908590492,Xiaomi Redmi Note 11s Dual Sim 128 Gb Azul Oca...,989900,NaN,47,Xiaomi,Nuevo,Note 11S,1,0.0000
2,MCO903010997,Xiaomi Mi 11t Pro Dual Sim 256 Gb Azul Celesti...,2248900,"3,699,900.0000",61,Xiaomi,Nuevo,11T Pro,1,5.0000
3,MCO951027990,Xiaomi Redmi Note 10 Pro (global) Dual Sim 128...,1117900,NaN,0,Xiaomi,Nuevo,Note 10 Pro (Global),1,0.0000
4,MCO864272471,Xiaomi Redmi 9c Dual Sim 64 Gb Gris Medianoch...,538900,NaN,49,Xiaomi,Nuevo,9C,1,0.0000
...,...,...,...,...,...,...,...,...,...,...
43,MCO621725027,iPhone 7 32gb 3 Meses De Uso Libre Factura Gar...,750000,NaN,4,Apple,Usado,iPhone 7,1,0.0000
44,MCO646379116,iPhone 7 128gb Negro Brillante,779900,NaN,1,Apple,Usado,iPhone 7,1,0.0000
45,MCO824231826,iPhone 8 64gb Celular Usado 5/5,829900,NaN,5,Apple,Usado,iPhone 8,1,0.0000
46,MCO617198289,Cajas iPhone 6s Con Manuales En Perfecto Estado,45000,NaN,1,Apple,Usado,iPhone 6s,0,0.0000


In [ ]:
items_mx

,id,title,price,original_price,sold_quantity,Marca,Condición del ítem,Modelo,free_shipping,rating
0,MLM1337805794,Samsung Galaxy A22 64 Gb Mint 4 Gb Ram,"3,799.0000","4,999.0000",2858,Samsung,Nuevo,A22,1,4.7000
1,MLM1430972053,Samsung Galaxy A53 5g Dual Sim 128 Gb Blanco A...,"6,518.0000","7,180.0000",147,Samsung,Nuevo,A53 5G Dual SIM,1,0.0000
2,MLM1453436209,Xiaomi Pocophone Poco M3 Pro 5g Dual Sim 128 G...,"4,408.8000",NaN,122,Xiaomi,Nuevo,Poco M3 Pro 5G,1,0.0000
3,MLM1365343604,Xiaomi Redmi 9a Dual Sim 64 Gb Azul Celeste 4 ...,"2,310.0000",NaN,7803,Xiaomi,Nuevo,9A,1,4.6000
4,MLM1475359479,Xiaomi Redmi 10c Dual Sim 128 Gb Verde Menta 4...,"3,059.0000","3,261.0000",651,Xiaomi,Nuevo,10C,1,4.6000
...,...,...,...,...,...,...,...,...,...,...
44,MLM1446092047,iPhone X 64gb De Exhibicion Originales Liberados,"6,380.0000",NaN,5,Apple,Usado,iPhone X,1,0.0000
45,MLM789199427,iPhone X 256gb De Exhibicion Originales Libera...,"7,280.0000",NaN,5,Apple,Usado,iPhone X,1,0.0000
46,MLM1365236801,Xiaomi Redmi 9a Dual Sim 64 Gb Azul Celeste 4 ...,"2,310.0000",NaN,250,Xiaomi,Nuevo,9A,1,4.1000
47,MLM1401500950,Xiaomi Redmi 9c Dual Sim 64 Gb Gris Medianoch...,"2,580.0000",NaN,5,Xiaomi,Nuevo,9C,1,0.0000


# 3. Data extraction for each phrase with trends tool

This process takes approximately 4 hours, because it takes a long time to extract the rating.

In [ ]:
def trends(site_id, category_id):
    url = 'https://api.mercadolibre.com/trends/'+site_id+'/'+category_id
    payload={}
    headers = {}
    response = requests.request("GET", url, headers=headers, data=payload, verify=False).json()
    df_trends = pd.DataFrame(response)
    df1 = pd.DataFrame()
    for i in range(len(df_trends['keyword'])):
        try:
            df = get_df(site_id, df_trends['keyword'][i], "q")
        except:
            continue
        df1 = pd.concat([df1, df])
        clear_output(wait=True)
        print("Load process: "+str((i/len(df_trends['keyword']))*100)+"%")
        df1 = df1.drop_duplicates()
    return df1

trends_co = trends("MCO", "MCO1055")
trends_mx = trends("MLM", "MLM1055")

In [ ]:
trends_co

In [ ]:
trends_mx

In [ ]:
trends_co.to_excel('trends_co.xlsx')
files.download('trends_co.xlsx')
trends_co.to_excel('trends_mx.xlsx')
files.download('trends_mx.xlsx')

In [ ]:
trends_co['from_trend']=1
items_co['from_trend']=0
trends_mx['from_trend']=1
items_mx['from_trend']=0

In [ ]:
base_co = pd.concat([trends_co, items_co]).drop_duplicates()
base_co.to_excel('base_co.xlsx')
files.download('base_co.xlsx')
base_mx = pd.concat([trends_mx, items_mx]).drop_duplicates()
base_mx.to_excel('base_mx.xlsx')
files.download('base_mx.xlsx')

# 5. Product reviews extraction

In [ ]:
base_co = pd.read_excel('base_co.xlsx')
#reviews = pd.read_excel('reviews.xlsx')
base_co = base_co[['id', 'title', 'price', 'original_price', 'sold_quantity',
       'Marca', 'Condición del ítem', 'Modelo', 'free_shipping', 'rating',
       'from_trend']].drop_duplicates()

def get_reviews_ra(ids):
    url_base = 'https://api.mercadolibre.com/reviews/item/'
    payload={}
    headers = {}  
    df = pd.DataFrame()
    for i in range(len(ids)):
        url = url_base+ids[i]
        response = requests.request("GET", url, headers=headers, data=payload, verify=False).json()
        try:
            df1 = pd.DataFrame(response['reviews'])
        except:
            continue
        clear_output(wait=True)
        print("General load process: "+str((i/len(ids))*100)+"%")
        df= pd.concat([df, df1])
    return df
reviews_co = get_reviews_ra(base_co['id'].tolist())
reviews_co

Proceso de carga general: 99.99244998112495%


,id,reviewable_object,date_created,status,title,content,rate,valorization,likes,dislikes,reviewer_id,buying_date,relevance,forbidden_words
0,151447302,"{'id': 'MCO875830062', 'type': 'product'}",2022-08-08T21:09:04Z,published,Poco recomendable. Peor que sus antecesores.,"Este celular es malísimo, me arrepiento de hab...",1,-1,5,6,4101125,2022-05-03T17:14:46Z,54,0
1,148555735,"{'id': 'MCO875830062', 'type': 'product'}",2022-07-08T14:57:03Z,published,Excelente,"Una excelente compra, llevo ya más de 2 semana...",5,0,0,0,115684066,2022-06-16T03:02:36Z,34,20
2,152160669,"{'id': 'MCO875830062', 'type': 'product'}",2022-08-16T16:33:14Z,published,Excelente,"Excelente, es mi 2do xiaomi, excelente resoluc...",5,0,0,0,53447378,2022-08-01T03:03:54Z,29,0
3,141255711,"{'id': 'MCO875830062', 'type': 'product'}",2022-04-25T19:41:19Z,published,Excelente,"Era lo que esperaba, espero y me dure bastant...",5,-2,4,6,42302814,2022-04-18T01:59:19Z,26,35
4,148730326,"{'id': 'MCO875830062', 'type': 'product'}",2022-07-10T18:31:40Z,published,Excelente,Voy por un mes con el celular y hasta el momen...,5,0,0,0,184683638,2022-06-12T17:30:50Z,20,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,144498615,"{'id': 'MCO655254556', 'type': 'item'}",2022-05-28T17:31:27Z,published,Sin igual !! Que belleza me fascino!!,Hermosisimo !! era lo q esperaba !!.,5,0,0,0,208477238,2022-05-27T12:44:28Z,5,0
0,148674551,"{'id': 'MCO610310767', 'type': 'item'}",2022-07-09T20:16:17Z,published,Muy bueno,"Producto interesante, tiene buen audio, la rec...",4,0,0,0,82385600,2022-06-01T21:10:42Z,23,0
1,139689454,"{'id': 'MCO610310767', 'type': 'item'}",2022-04-08T19:15:58Z,published,Muy bueno,Es bueno pero deberia traer instrucciones de m...,4,0,0,0,282145430,2022-03-24T19:52:05Z,13,0
2,153273747,"{'id': 'MCO610310767', 'type': 'item'}",2022-08-27T00:17:02Z,published,Excelente,"Me encantó, super dis reto, buena señal y func...",5,0,0,0,96961800,2022-07-25T18:54:51Z,9,0


In [ ]:
base_mx = pd.read_excel('base_mx.xlsx')
base_mx = base_mx[['id', 'title', 'price', 'original_price', 'sold_quantity',
       'Marca', 'Condición del ítem', 'Modelo', 'free_shipping', 'rating',
       'from_trend']].drop_duplicates()
reviews_mx = get_reviews_ra(base_mx['id'].tolist())

Proceso de carga general: 99.99309630652398%


In [ ]:
reviews_mx

,id,reviewable_object,date_created,status,title,content,rate,valorization,likes,dislikes,reviewer_id,buying_date,relevance,forbidden_words
0,129816983,"{'id': 'MLM1348021632', 'type': 'product'}",2021-12-28T02:05:43Z,published,Excelente,"Muy bueno, sin fallas, sin ponerse lento. El d...",5,2,3,1,44861575,2021-12-10T18:46:33Z,18,0
1,129247994,"{'id': 'MLM1348021632', 'type': 'product'}",2021-12-21T21:58:18Z,published,Recomendado tiene lo que se requiere,"Mejor de lo que esperaba, cumple con mis expe...",5,1,1,0,127947549,2021-12-03T01:14:11Z,9,0
2,139913365,"{'id': 'MLM1348021632', 'type': 'product'}",2022-04-11T17:49:41Z,published,Excelente,Muy buen producto. Justo lo que dice ser.,5,0,0,0,570208695,2021-12-11T07:19:24Z,8,0
3,131320889,"{'id': 'MLM1348021632', 'type': 'product'}",2022-01-10T19:40:40Z,published,Excelente,3b's vale la pena tener este celular.,5,0,0,0,399913944,2021-12-12T04:39:25Z,8,0
4,153282194,"{'id': 'MLM1348021632', 'type': 'product'}",2022-08-27T02:13:06Z,published,Excelente,Muy buen articulo completamente nuevo.,5,0,0,0,46425583,2022-08-12T01:19:40Z,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,146429574,"{'id': 'MLM1376085949', 'type': 'product'}",2022-06-17T03:14:20Z,published,Excelente teléfono gaming y muy bonito,Excelente teléfono lo uso para juegos y va de ...,5,0,0,0,82056998,2022-06-05T07:31:26Z,64,0
1,148056427,"{'id': 'MLM1376085949', 'type': 'product'}",2022-07-02T22:21:38Z,published,Muy bueno,Al principio me espante porque se me descargo ...,4,0,0,0,294548437,2022-06-16T23:13:37Z,63,0
2,146147607,"{'id': 'MLM1376085949', 'type': 'product'}",2022-06-14T18:20:59Z,published,Buena compra,"Me gusta mucho, rápido, buena pantalla, cámara...",5,0,0,0,65285333,2022-05-30T18:11:46Z,40,20
3,150018819,"{'id': 'MLM1376085949', 'type': 'product'}",2022-07-24T14:56:45Z,published,Bueno,"La relación de precio y calidad es excelente, ...",4,0,0,0,131465992,2022-05-31T15:27:35Z,32,0


In [ ]:
reviews1_mx = reviews_mx
reviews1_mx['reviewable_object']=reviews1_mx['reviewable_object'].astype(str).str.slice(8, 21)
reviews1_mx

,id,reviewable_object,date_created,status,title,content,rate,valorization,likes,dislikes,reviewer_id,buying_date,relevance,forbidden_words
0,129816983,MLM1348021632,2021-12-28T02:05:43Z,published,Excelente,"Muy bueno, sin fallas, sin ponerse lento. El d...",5,2,3,1,44861575,2021-12-10T18:46:33Z,18,0
1,129247994,MLM1348021632,2021-12-21T21:58:18Z,published,Recomendado tiene lo que se requiere,"Mejor de lo que esperaba, cumple con mis expe...",5,1,1,0,127947549,2021-12-03T01:14:11Z,9,0
2,139913365,MLM1348021632,2022-04-11T17:49:41Z,published,Excelente,Muy buen producto. Justo lo que dice ser.,5,0,0,0,570208695,2021-12-11T07:19:24Z,8,0
3,131320889,MLM1348021632,2022-01-10T19:40:40Z,published,Excelente,3b's vale la pena tener este celular.,5,0,0,0,399913944,2021-12-12T04:39:25Z,8,0
4,153282194,MLM1348021632,2022-08-27T02:13:06Z,published,Excelente,Muy buen articulo completamente nuevo.,5,0,0,0,46425583,2022-08-12T01:19:40Z,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,146429574,MLM1376085949,2022-06-17T03:14:20Z,published,Excelente teléfono gaming y muy bonito,Excelente teléfono lo uso para juegos y va de ...,5,0,0,0,82056998,2022-06-05T07:31:26Z,64,0
1,148056427,MLM1376085949,2022-07-02T22:21:38Z,published,Muy bueno,Al principio me espante porque se me descargo ...,4,0,0,0,294548437,2022-06-16T23:13:37Z,63,0
2,146147607,MLM1376085949,2022-06-14T18:20:59Z,published,Buena compra,"Me gusta mucho, rápido, buena pantalla, cámara...",5,0,0,0,65285333,2022-05-30T18:11:46Z,40,20
3,150018819,MLM1376085949,2022-07-24T14:56:45Z,published,Bueno,"La relación de precio y calidad es excelente, ...",4,0,0,0,131465992,2022-05-31T15:27:35Z,32,0


In [ ]:
reviews1_co = reviews_co
reviews1_co['reviewable_object']=reviews1_co['reviewable_object'].astype(str).str.slice(8, 20)
reviews1_co

In [ ]:
reviews1.to_excel('reviews_mx.xlsx')
files.download('reviews_mx.xlsx')
reviews1.to_excel('reviews_co.xlsx')
files.download('reviews_co.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>